In [4]:
import os
print(os.listdir())


['.config', '.ipynb_checkpoints', 'Audio_Dataset', 'sample_data']


In [ ]:
!pip install librosa tensorflow tensorflow-hub pandas scikit-learn


In [10]:
import librosa, numpy as np, tensorflow_hub as hub, pandas as pd

# Load YAMNet
yamnet_model = hub.load("https://tfhub.dev/google/yamnet/1")

# Load class labels
class_map_url = "https://raw.githubusercontent.com/tensorflow/models/master/research/audioset/yamnet/yamnet_class_map.csv"
class_names = pd.read_csv(class_map_url)["display_name"].tolist()

def recognize_sound(file_path):
    # Load your mp3, resample to 16kHz mono
    wav, sr = librosa.load(file_path, sr=16000, mono=True)
    # Run through YAMNet
    scores, embeddings, spectrogram = yamnet_model(wav)
    mean_scores = np.mean(scores, axis=0)
    top_class = np.argmax(mean_scores)
    return class_names[top_class], float(mean_scores[top_class])

#  Test on your gunshot file
label, conf = recognize_sound("Audio_Dataset/machine-gun.mp3")
print(f"Detected: {label} (confidence={conf:.2f})")


Detected: Silence (confidence=0.50)


In [11]:
results = []
for file in os.listdir("Audio_Dataset"):
    if file.endswith(".mp3") or file.endswith(".wav"):
        filepath = os.path.join("Audio_Dataset", file)
        label, conf = recognize_sound(filepath)
        results.append({"file": file, "label": label, "confidence": conf})

import pandas as pd
df = pd.DataFrame(results)
print(df)


                    file             label  confidence
0        machine-gun.mp3           Silence    0.498635
1            gunfire.mp3  Gunshot, gunfire    0.614205
2  machine-gun-burst.mp3  Gunshot, gunfire    0.998877
3            gunload.mp3        Mechanisms    0.307474
4     single-gunshot.mp3         Explosion    0.529216
